In [1]:
from tensorflow.keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras import Sequential
from data import Data

In [2]:
data = Data("data")
data.config_iter(64, 3)
len(data)

4

In [17]:
graphCNN = Sequential([
    Input((256, 256, 1)),
    Conv2D(8, (3, 3), activation="relu", padding="same"),
    Conv2D(16, (3, 3), activation="relu", padding="same"),
    Conv2D(16, (3, 3), activation="relu", strides=(2, 2), padding="same"),  # 128 128 32
    Conv2D(32, (3, 3), activation="relu", padding="same"),
    Conv2D(32, (3, 3), activation="relu", strides=(2, 2), padding="same"),  # 64 64 64
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    Conv2D(64, (3, 3), activation="relu", strides=(2, 2), padding="same"),  # 32 32 128
    Conv2D(128, (3, 3), activation="relu", padding="same"),
    Conv2D(256, (3, 3), activation="relu", strides=(2, 2), padding="same"),  # 16 16 256
    Conv2D(128, (3, 3), activation="relu", padding="same"),  # 16 16 128
])

embedding_way = Sequential([
    Input((16, 16, 1)),
    Flatten(),
    Dense(256, "relu"),
    RepeatVector(graphCNN.output_shape[3]),
    Reshape(graphCNN.output_shape[1:]),
])

inp1 = Input((256, 256, 1))
inp2 = Input((16, 16, 1))

res_grpah = graphCNN(inp1) # 128,
res_way = embedding_way(inp2)


cont = concatenate([
    res_grpah,
    res_way
])  # 16 16 256






<KerasTensor: shape=(None, 16, 16, 256) dtype=float32 (created by layer 'concatenate_12')>